In [32]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from lightgbm import LGBMRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor

os.chdir('C:/Users/kksp1/OneDrive/Desktop/3_2학기 해야할 일/수업/3-2/머신러닝/미니프로젝트/풀젝_3')
#print(os.getcwd())
input_file = "kc_house_data.csv"
housing_train = pd.read_csv(input_file)

## 꼬리표 나누기

In [33]:
housing_train = housing_train.drop("id", axis=1)
# housing_train = housing_train.drop("zipcode", axis=1)
train_target = housing_train["price"].copy()


In [34]:
ordinal_encoder = OrdinalEncoder() #각 범주를 대응하는 숫자로 변환

## 특성공학

In [35]:
ordinal_encoder = OrdinalEncoder()
housing_train['date'] = pd.to_datetime(housing_train['date'])
housing_train["tr_year"] = housing_train["date"].dt.year
housing_train["tr_month"] = housing_train["date"].dt.month
housing_train["tr_day_name"] = housing_train["date"].dt.day_name
housing_train["date"] = housing_train["date"].dt.strftime('%Y-%m')
housing_train["tr_year"] = ordinal_encoder.fit_transform(housing_train[["tr_year"]])
housing_train["tr_month"] = ordinal_encoder.fit_transform(housing_train[["tr_month"]])
housing_train["tr_day_name"] = ordinal_encoder.fit_transform(housing_train[["tr_day_name"]])
housing_train["date"] = ordinal_encoder.fit_transform(housing_train[["date"]])


### 데이터 표준화

In [36]:
scaler = StandardScaler()
scaler.fit_transform(housing_train)

array([[-0.12093399, -0.86671733, -0.39873715, ..., -0.69065478,
         1.09962055,  0.        ],
       [ 0.41670797, -0.00568792, -0.39873715, ..., -0.69065478,
         1.74162654,  0.        ],
       [ 0.95434994, -0.98084935, -1.47395936, ...,  1.44790136,
        -1.46840343,  0.        ],
       ...,
       [-1.19621792, -0.37586519, -1.47395936, ..., -0.69065478,
        -0.18439144,  0.        ],
       [ 0.68552895, -0.38158814, -0.39873715, ...,  1.44790136,
        -1.78940643,  0.        ],
       [-0.12093399, -0.58588173, -1.47395936, ..., -0.69065478,
         1.09962055,  0.        ]])

### 특성공학 이어서

In [37]:
housing_train['sqft_living+bathrooms'] = housing_train['sqft_living'] + housing_train['bathrooms']
# housing_train['house_age'] = 2023 - housing_train['yr_built']
# housing_train['years_since_renovation'] = 2023 - housing_train['yr_renovated']
# housing_train['indoor_to_outdoor_ratio'] = housing_train['sqft_living'] / housing_train['sqft_lot']
housing_train['basement_to_living_ratio'] = housing_train['sqft_basement'] / housing_train['sqft_living']
housing_train['location'] = housing_train['lat'] + housing_train['long']

# housing_train['bedrooms_to_bathrooms_ratio'] = housing_train['bedrooms'] / housing_train['bathrooms']
housing_train['living_to_floors_ratio'] = housing_train['sqft_living'] / housing_train['floors']

housing_train['Totalviews'] = housing_train['view'] + housing_train['waterfront']
housing_train['Totalviews//'] = housing_train['view'] // housing_train['waterfront']

housing_train['Toalsizes'] = housing_train['sqft_living'] + housing_train['sqft_above'] + housing_train['sqft_basement'] + housing_train['sqft_living15']



housing_train["house_per_sqft_livings"] = 1 / (housing_train["sqft_living"] + housing_train["sqft_living15"])

housing_train['(sqft_living*grade)/2'] = (housing_train['sqft_living'] * housing_train['grade']) / 2


housing_train['(sqft_living//waterfront/sqft_living)'] = ((housing_train['sqft_living']))//(housing_train['waterfront']/housing_train['sqft_living'])


housing_train["sqft_living*waterfront"] = housing_train["sqft_living"] * housing_train["waterfront"]


# #삭제될 예정
# housing_train['(sqft_living//waterfront/sqft_living)'] = ((housing_train['sqft_living']))//(housing_train['waterfront']/housing_train['sqft_living'])
housing_train['(waterfront//waterfront/sqft_living)'] = (housing_train['waterfront']//(housing_train['waterfront']/housing_train['sqft_living']))
housing_train['((sqft_living//waterfront/sqft_living)//(waterfront//waterfront/sqft_living))'] = housing_train['(sqft_living//waterfront/sqft_living)']//housing_train['(waterfront//waterfront/sqft_living)']
# #삭제될 예정
housing_train['(waterfront/waterfront/sqft_living)'] = (housing_train['waterfront']/housing_train['waterfront']/housing_train['sqft_living'])
housing_train['(sqft_living**2)//(waterfront/waterfront/sqft_living)'] = (housing_train['sqft_living']**2)//(housing_train['(waterfront/waterfront/sqft_living)'])
#((sqft_living**2)//(waterfront//(waterfront/sqft_living)))

# #삭제될 예정
housing_train['(waterfront//(waterfront/sqft_living))'] = housing_train['waterfront']//(housing_train['waterfront']/housing_train['sqft_living'])#(sqft_living//((sqft_living//waterfront/sqft_living)//(waterfront//waterfront/sqft_living)) ## 89.7
housing_train['(sqft_living//((sqft_living//waterfront/sqft_living)//(waterfront//waterfront/sqft_living))'] = ((housing_train['sqft_living']) // ((housing_train['(sqft_living//waterfront/sqft_living)'])) // (housing_train['(waterfront//waterfront/sqft_living)']))

# #사용될 예정
housing_train['(waterfront//waterfront/sqft_living/(sqft_living//((sqft_living//waterfront/sqft_living)//(waterfront//waterfront/sqft_living))))'] = housing_train['(waterfront//waterfront/sqft_living)'] / housing_train['(sqft_living//((sqft_living//waterfront/sqft_living)//(waterfront//waterfront/sqft_living))']
housing_train['((sqft_living**2)//(waterfront//(waterfront/sqft_living)))'] = ((housing_train['sqft_living']**2)//(housing_train['waterfront']//(housing_train['waterfront']/housing_train['sqft_living'])))
# ((sqft_living**2)//(waterfront//waterfront/sqft_living))                         0.889701
# (waterfront//waterfront/sqft_living/(sqft_living//((sqft_living//waterfront/sqft_living)//(waterfront//waterfront/sqft_living)))) # 89.7

# # 0.911651 ##
housing_train['(((waterfront//waterfront/sqft_living))/((((waterfront//waterfront/sqft_living))//(((sqft_living**2)//(waterfront//(waterfront/sqft_living)))))))']= (((housing_train['(waterfront//waterfront/sqft_living)'])/((((housing_train['(waterfront//waterfront/sqft_living)']))//(housing_train['((sqft_living**2)//(waterfront//(waterfront/sqft_living)))'])))))


# # # 0.912095
# housing_train['(sqft_living//waterfront/(((sqft_living**2)//(waterfront//(waterfront/sqft_living)))//(((waterfront//waterfront/sqft_living))/((((waterfront//waterfront/sqft_living))//(((sqft_living**2)//(waterfront//(waterfront/sqft_living)))))))))']= housing_train['sqft_living//waterfront']/(housing_train['((sqft_living**2)//(waterfront//(waterfront/sqft_living)))']//housing_train['(((waterfront//waterfront/sqft_living))/((((waterfront//waterfront/sqft_living))//(((sqft_living**2)//(waterfront//(waterfront/sqft_living)))))))'])

# 0.928359

# housing_train['(3*location+(((waterfront//waterfront/sqft_living))/((((waterfront//waterfront/sqft_living))//(((sqft_living**2)//(waterfront//(waterfront/sqft_living))))))))'] = (3*housing_train['location'])+housing_train['(((waterfront//waterfront/sqft_living))/((((waterfront//waterfront/sqft_living))//(((sqft_living**2)//(waterfront//(waterfront/sqft_living)))))))']

# housing_train['(2*location+(((waterfront//waterfront/sqft_living))/((((waterfront//waterfront/sqft_living))//(((sqft_living**2)//(waterfront//(waterfront/sqft_living))))))))'] = (2*housing_train['location'])+housing_train['(((waterfront//waterfront/sqft_living))/((((waterfront//waterfront/sqft_living))//(((sqft_living**2)//(waterfront//(waterfront/sqft_living)))))))']

# housing_train['(3*lat+(((waterfront//waterfront/sqft_living))/((((waterfront//waterfront/sqft_living))//(((sqft_living**2)//(waterfront//(waterfront/sqft_living))))))))'] = (3*housing_train['lat']) + housing_train['(((waterfront//waterfront/sqft_living))/((((waterfront//waterfront/sqft_living))//(((sqft_living**2)//(waterfront//(waterfront/sqft_living)))))))']


housing_train = housing_train.drop('(waterfront//waterfront/sqft_living)', axis=1)
housing_train = housing_train.drop('(waterfront/waterfront/sqft_living)', axis=1)
housing_train = housing_train.drop('(sqft_living**2)//(waterfront/waterfront/sqft_living)', axis=1)
housing_train = housing_train.drop('(sqft_living//((sqft_living//waterfront/sqft_living)//(waterfront//waterfront/sqft_living))', axis=1)
housing_train = housing_train.drop('(waterfront//waterfront/sqft_living/(sqft_living//((sqft_living//waterfront/sqft_living)//(waterfront//waterfront/sqft_living))))', axis=1)
housing_train = housing_train.drop('(sqft_living//waterfront/sqft_living)', axis=1)


#((sqft_living+2waterfront//waterfront/sqft_living)/(waterfront//((sqft_living//waterfront/sqft_living)//(waterfront//waterfront/sqft_living))))    -0.834290
#                                                                                                                                                     (               (waterfront//waterfront/sqft_living)  /(               waterfront  //               ((sqft_living//waterfront/sqft_living)//(waterfront//waterfront/sqft_living))  ))                 -0.833450
# housing_train['((waterfront//waterfront/sqft_living)/(waterfront//((sqft_living//waterfront/sqft_living)//(waterfront//waterfront/sqft_living))))'] = (housing_train['(waterfront//(waterfront/sqft_living))']/(housing_train['waterfront']//housing_train['((sqft_living//waterfront/sqft_living)//(waterfront//waterfront/sqft_living))']))
housing_train = housing_train.drop('((sqft_living//waterfront/sqft_living)//(waterfront//waterfront/sqft_living))', axis=1)
housing_train = housing_train.drop('(waterfront//(waterfront/sqft_living))', axis=1)


filtered_train_data = housing_train.drop('price', axis=1)
# filtered_train_data = filtered_train_data.drop('tr_year', axis=1)
filtered_train_data.replace([np.inf, -np.inf], np.nan, inplace=True)


## 함수정의

In [38]:

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

## 모델 선정

In [29]:
from sklearn.model_selection import cross_val_score
# 기본 시드 설정
seed = 21345

# 데이터 섞기
np.random.seed(seed)
sh_in = np.random.permutation(len(filtered_train_data))
sh_t_data = filtered_train_data.values[sh_in]
sh_t_target = train_target.values[sh_in]

lgbm = LGBMRegressor(max_depth = None, min_data_in_leaf = 20, feature_fraction = 1.0, n_estimators=100,
                     bagging_fraction = 0.1, min_gain_to_split = 1, Task = 1,
                     application = 'regression',num_boost_round = 790,learning_rate = 0.1155,
                     num_leaves = 31 ,random_state=0)

lgbm_scores = cross_val_score(lgbm, sh_t_data, sh_t_target, scoring="neg_mean_squared_error", cv = 10, n_jobs=-1)
lgbm_rmse_scores = np.sqrt(-lgbm_scores)

print("=======================================================================================")
display_scores(lgbm_rmse_scores)
print("=======================================================================================")

Scores: [104587.68152537 103300.76822701 113641.87072352 100002.08872727
 109321.91751173 118264.85898364 111451.19089265 114809.68031665
 118500.5677847  115322.09727176]
Mean: 110920.27219643048
Standard deviation: 6111.598664147609


In [40]:
# 기본 시드 설정
seed = 21366

# 데이터 섞기
np.random.seed(seed)
sh_in = np.random.permutation(len(filtered_train_data))
sh_t_data = filtered_train_data.values[sh_in]
sh_t_target = train_target.values[sh_in]

catboost = CatBoostRegressor(depth=7, learning_rate=0.1, n_estimators=2000, verbose=0)

catboost_scores = cross_val_score(catboost, sh_t_data, sh_t_target, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)
catboost_rmse_scores = np.sqrt(-catboost_scores)

print("=======================================================================================")
display_scores(catboost_rmse_scores)
print("=======================================================================================")

Scores: [122348.05952307  94499.35636369 117007.53506535  83628.8717908
 105830.95531101 121477.34595344 105426.34835928  96323.97220175
 106788.33147607 111901.67908371]
Mean: 106523.24551281861
Standard deviation: 11778.802518379614


# Mean: 106523.24551281861